In [ ]:
#뉴스데이터 불러오기
import pickle
import pandas as pd

# with open('naver_data.pkl', 'rb') as f:
#     data1 = pickle.load(f)
#댓글 데이터 불러오기
data2 = pd.read_csv('naver_comments_all.csv')
data2.head(2)

In [ ]:
# 데이터 그룹화
data4 = data2.groupby('url').nunique()
data4

In [ ]:
#벡터화
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

# URL별 유니크한 user_id들을 공백으로 연결
data3_str = data3.apply(lambda x: ' '.join(x))

# Counter Vectorization
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data3_str)

# LSA (Latent Semantic Analysis)
svd = TruncatedSVD(n_components=10)  # n_components는 잠재 변수의 수, 적절한 값을 선택해야 합니다.
lsa_result = svd.fit_transform(X)

print(lsa_result)


In [ ]:
#코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity

# LSA 결과에서 코사인 유사도 계산
cosine_sim = cosine_similarity(lsa_result)

# URL과 인덱스 매핑 딕셔너리 생성
indices = pd.Series(range(len(data3_str.index)), index=data3_str.index)



In [ ]:
# 기사링크 기반 추천시스템
def recommend(url, cosine_sim=cosine_sim):
    # 선택한 URL의 인덱스 가져오기
    idx = indices[url]

    # 모든 URL에 대해 해당 URL과의 유사도 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 URL들 정렬하기
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 URL 가져오기 (자신 제외)
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 URL 인덱스 가져오기
    url_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 URL 반환하기
    return data3_str.index[url_indices]

In [ ]:
#코사인유사도
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# 데이터 전처리: 각 user_id별로 방문한 url들을 공백으로 연결
df2_str_user = data2.groupby('user_id')['url'].apply(lambda x: ' '.join(x.values))
# Counter Vectorization
vectorizer = CountVectorizer()
X_user = vectorizer.fit_transform(df2_str_user)
# LSA (Latent Semantic Analysis)
svd = TruncatedSVD(n_components=10)  # n_components는 잠재 변수의 수, 적절한 값을 선택해야 합니다.
lsa_result_user = svd.fit_transform(X_user)
# User ID와 인덱스 매핑 딕셔너리 생성
indices_user = pd.Series(range(len(df2_str_user.index)), index=df2_str_user.index)


In [ ]:
# 데이터 전처리: 각 URL별로 방문한 user들을 공백으로 연결
df2_str_url = data2.groupby('url')['user_id'].apply(lambda x: ' '.join(x.values))
# Counter Vectorization
vectorizer = CountVectorizer()
X_url = vectorizer.fit_transform(df2_str_url)
# LSA (Latent Semantic Analysis)
svd = TruncatedSVD(n_components=10)  # n_components는 잠재 변수의 수, 적절한 값을 선택해야 합니다.
lsa_result_url = svd.fit_transform(X_url)


In [ ]:
def recommend_urls_for_users(user_id):
    # 선택된 user의 인덱스 가져오기
    idx = indices_user[user_id]
    # 모든 URL에 대해 해당 user와의 유사도 가져오기
    sim_scores_url = cosine_similarity(lsa_result_user[idx].reshape(1,-1), lsa_result_url)
    # 유사도에 따라 url 정렬하기 (내림차순)
    sim_scores_url_sorted_indices = np.argsort(-sim_scores_url.flatten())
    # 가장 유사한 10개의 url 가져오기
    top_5_similar_urls_indices = sim_scores_url_sorted_indices[:5]
    
    return data3_str.index[top_5_similar_urls_indices]


In [ ]:
#출력확인
# dict2[recommend_urls_for_users('tvRC')[0]]

In [ ]:
dict2 = {url: title for title, url in zip(data2.title,data2.url)}
# dict2

In [ ]:
#딕셔너리 생성
# dict1 = {name: id for id, name in zip(data2.user_id, data2.user_name)}
dict1.keys()

In [ ]:
#아이디를 입력받아 뉴스 추천5개op
a = input("아이디를 입력하세요 : ")
for i in range(5):
    print(recommend_urls_for_users(dict1[a[:4] + '****'])[i])


In [ ]:
list1 = list(dict1)


In [ ]:
# list_all = []
# for i in list1:
#     user_key = i[:4] + '****'
#     try:
#         for j in range(5):
#             recommended_urls = dict2[recommend_urls_for_users(dict1[user_key])[j]]
#             list_all.append(recommended_urls)
#     except KeyError:
#         pass



In [ ]:
# import pandas as pd

# list_all = []
# for i in list1:
#     user_key = i[:4] + '****'
#     try:
#         for j in range(5):
#             recommended_url = dict2[recommend_urls_for_users(dict1[user_key])[j]]
#             list_all.append(recommended_url)
#     except KeyError:
#         pass

# df = pd.DataFrame(list_all, columns=['recommended_url'])
# df.to_csv('output.csv', index=False)
